### Ch5.1 계산 그래프 (Computational graph)
- 국소적 계산 : 전체에서 어떤 일이 벌어지든 상관없이 자신과 관계된 정보만으로 결과를 출력할 수 있음
- 연쇄법칙(Chain rule) : 여러 함수로 구성되어 있는 함수인 합성 함수의 미분에 대한 성질
- 합성 함수의 미분 : 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있음

#### 단순한 계층 구현하기

In [1]:
### 곱셉 계층
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy
    

apple = 100
apple_num = 2
tax = 1.1

### 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

### 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print(price)

### 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [2]:
### 덧셈 계층
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    
    
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
all_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)  #(!)
orange_price = mul_orange_layer.forward(orange, orange_num)  #(2)
all_price = all_apple_orange_layer.forward(apple_price, orange_price)  #(3)
price = mul_tax_layer.forward(all_price, tax)  #(4)
print(price)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  #(4)
dapple_price, dorange_price = all_apple_orange_layer.backward(dall_price)  #(3)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  #(2)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  #(!)
print(dapple, dapple_num, dorange, dorange_num, tax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 1.1


#### 활성화 함수 계층 구현하기

In [3]:
import numpy as np

### ReLU 계층
class ReLU:
    def __init__(self):
        self.mask = None   ### mask는 True/False로 구성된 넘파이 배열
        
    def forward(self, x):
        self.mask = (x <= 0)   ### x의 원소값이 0이하인 인덱스는 True, 그 외는 False
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
    
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [4]:
### Sigmoid 계층
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

#### Affine/Softmax 계층 구현하기

In [5]:
### Affine 계층
class Affine:
    def __init__(self):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [6]:
### Softmax-with-Loss 계층 : Softmax 계층과 손실함수인 Cross Entropy Error(교차 엔트로피 오차)도 포함
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None   # 손실
        self.y = None   # softmax의 출력
        self.t = None   # 정답레이블(one-hot vector)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

#### 오차역전파법 구현하기

- TwoLayerNet 클래스의 인스턴스 변수
1. params: 딕셔너리 변수로, 신경망의 매개변수를 보관
2. layers: 순서가 있는 딕셔너리 변수로, 신경망의 계층을 보관
3. lastLayer: 신경망의 마지막 계층
- TwoLayerNet 클래스의 매서드
1. __init__(self, input_size, hidden_size, output_size, weight_init_std): 초기화를 수행
2. predict(self, x): 예측(추론)을 수행
3. loss(self, x, t): 손실 함수의 값을 구함
4. accuracy(self, x, t): 정확도를 구함
5. numerical_gradient(self, x ,t): 가중치 매개변수의 기울기를 수치미분방식으로 구함
6. gradient(self, x, t): 가중치 매개변수의 기울기를 오차역전파법으로 구함

In [7]:
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        ### 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        ### 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['ReLU1'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y ,t)
    
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: 
            t = np.argmax(t, axis=1)    
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    
    def gradient(self, x, t):
        ### 순전파
        self.loss(x, t)
        
        ### 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        ### 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

#### 오차역전파법으로 구한 기울기 검증하기
: 수치미분의 결과와 오차역전파법의 결과를 비교하여 오차역전파법을 제대로 구현했는지 검증

In [8]:
from dataset.mnist import load_mnist

### 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50,output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

### 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:1.977636552801595e-08
b1:2.876046944908254e-07
W2:1.0018366839375593e-12
b2:1.1990409776174716e-10


#### 오차역전파법을 사용한 학습 구현하기

In [9]:
from dataset.mnist import load_mnist

### 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50,output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    ### 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    ### 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.11808333333333333 0.1219
0.9037166666666666 0.9066
0.9231666666666667 0.9241
0.9326333333333333 0.9327
0.9426666666666667 0.9419
0.94905 0.949
0.9539666666666666 0.95
0.9592333333333334 0.9565
0.9626 0.9593
0.9650666666666666 0.96
0.9676 0.9626
0.96965 0.9643
0.9727666666666667 0.9655
0.9742666666666666 0.9655
0.9758333333333333 0.9683
0.9763166666666667 0.9684
0.9778333333333333 0.9683
